In [64]:
import pandas as pd
import numpy as np
from shapely import wkt, geometry
import pydrive
import PIL
from PIL import Image
import os
import shutil

import cv2 as cv

import torch
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode

import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

logging.basicConfig(format='%(asctime)s %(levelname)-4s %(message)s',
                    level=logging.INFO,
                    datefmt='%d-%m-%Y %H:%M:%S')

logging.getLogger("PIL").setLevel(logging.WARNING)
logging.getLogger("shapely").setLevel(logging.WARNING)
logging.getLogger("pydrive").setLevel(logging.WARNING)

PIL.Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
metadata = pd.read_csv("../../os_meta.csv")

In [3]:
nunique = metadata.nunique()
repeated_col_data = nunique[nunique == 1].index
meta = metadata.drop(repeated_col_data, axis = 1, inplace = False)
meta = meta[["WKT", "SHEET_NO", "IMAGE", "IMAGEURL", "YEAR"]]
meta["WKT"] = meta["WKT"].apply(lambda x: wkt.loads(x).bounds)
meta[["minx", "miny", "maxx", "maxy"]] = pd.DataFrame(meta["WKT"].tolist(), index = meta.index)
meta.drop("WKT", axis = 1, inplace = True)

In [4]:
meta

,SHEET_NO,IMAGE,IMAGEURL,YEAR,minx,miny,maxx,maxy
0,007_05,82877892,https://maps.nls.uk/view/82877892,1894,-3.305799,55.886052,-3.267098,55.900605
1,007_08,82877928,https://maps.nls.uk/view/82877928,1894,-3.190029,55.886313,-3.151372,55.900830
2,007_09,82877940,https://maps.nls.uk/view/82877940,1894,-3.305674,55.871598,-3.266988,55.886151
3,007_10,82877949,https://maps.nls.uk/view/82877949,1894,-3.267098,55.871697,-3.228427,55.886238
4,007_11,82877958,https://maps.nls.uk/view/82877958,1894,-3.228523,55.871785,-3.189866,55.886313
...,...,...,...,...,...,...,...,...
207,008_01,82878021,https://maps.nls.uk/view/82878021,1934,-3.151507,55.900830,-3.112850,55.915335
208,008_02,82878033,https://maps.nls.uk/view/82878033,1934,-3.112903,55.900881,-3.074261,55.915374
209,008_05,82878063,https://maps.nls.uk/view/82878063,1934,-3.151439,55.886376,-3.112798,55.900881
210,008_06,82878075,https://maps.nls.uk/view/82878075,1934,-3.112850,55.886427,-3.074222,55.900920


In [5]:
print(meta["minx"].min())
print(meta["maxx"].max())
print(meta["miny"].min())
print(meta["maxy"].max())

-3.38374788940603
-3.07422210721001
55.8715983250667
56.0020051346329


In [8]:
same_area_meta = metadata[["WKT", "IMAGE"]]

In [11]:
grouped_area = same_area_meta.groupby("WKT")
grouped_area = grouped_area["IMAGE"].apply(list).reset_index()
grouped_area_4 = grouped_area[grouped_area["IMAGE"].apply(lambda x: len(x) == 4)]
grouped_area_4 = grouped_area_4.assign(WKT=grouped_area_4["WKT"].apply(lambda x: wkt.loads(x).bounds))
grouped_area_4[["minx", "miny", "maxx", "maxy"]] = pd.DataFrame(grouped_area_4["WKT"].tolist(), index = grouped_area_4.index)
grouped_area_4.drop("WKT", axis = 1, inplace = True)
grouped_area_4

,IMAGE,minx,miny,maxx,maxy
0,"[82878066, 82878069, 82878072, 82878075]",-3.112850,55.886427,-3.074222,55.900920
1,"[82878024, 82878027, 82878030, 82878033]",-3.112903,55.900881,-3.074261,55.915374
2,"[82877628, 82877631, 82877634, 82877637]",-3.112957,55.915335,-3.074300,55.929828
3,"[82877583, 82877586, 87213123, 82877589]",-3.113011,55.929788,-3.074339,55.944281
4,"[82877541, 82877544, 82877547, 82877550]",-3.113064,55.944242,-3.074378,55.958735
6,"[82878054, 82878057, 82878060, 82878063]",-3.151439,55.886376,-3.112798,55.900881
7,"[82878012, 82878015, 82878018, 82878021]",-3.151507,55.900830,-3.112850,55.915335
8,"[82877610, 82877613, 82877616, 82877619]",-3.151575,55.915284,-3.112903,55.929788
9,"[82877571, 82877574, 82877577, 82877580]",-3.151644,55.929737,-3.112957,55.944242
10,"[82877529, 82877532, 82877535, 82877538]",-3.151712,55.944191,-3.113011,55.958696


In [23]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [24]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=278936521475-rubbm8v9q9lpr99h8n9onm0pgh2hbqq5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [48]:
drive = GoogleDrive(gauth)

In [62]:
MAP_DIRECTORY = "../data/originals/"

def download_map_from_gdrive(map_name):
    drive_file_name = f"{map_name}.27.tif"
    
    logging.info(f"Fetching file {drive_file_name}")
    file_list = drive.ListFile({'q': f"'13favbDFNNPK78LNSIiWRCzNyhFtYe1Ix' in parents and trashed=false and title = '{drive_file_name}'"}).GetList()
    
    if len(file_list) == 0:
        raise ValueError(f"The file {map_name}.27.tif wasn't found in the shared drive")
    else:
        file_id = file_list[0]["id"]
        file = drive.CreateFile({'id': file_id})
        file_path = os.path.join(MAP_DIRECTORY, map_name) + ".tif"
        logging.info(f"Downloading file {drive_file_name}")
        file.GetContentFile(file_path)
        logging.info(f"Done!")
        
        return file_path

def rotate_crop_map(map_img):
    gray = cv.cvtColor(map_img, cv.COLOR_RGB2GRAY)
    gray = 255 - gray
    thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]

    # Compute rotated bounding box
    coords = np.column_stack(np.where(thresh > 0))
    center_rect, dims, angle = cv.minAreaRect(coords)

    if angle < -45:
        angle = -angle
    else:
        angle = 90-angle

    # Rotate image to deskew
    (h, w) = map_img.shape[:2]
    center = (w // 2, h // 2)
    M = cv.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv.warpAffine(map_img, M, (w, h), flags=cv.INTER_CUBIC, borderMode=cv.BORDER_REPLICATE)

    gray = cv.cvtColor(rotated, cv.COLOR_BGR2GRAY)
    gray = 255*(gray < 128).astype(np.uint8) # To invert the text to white
    coords = cv.findNonZero(gray) # Find all non-zero points (text)
    x, y, w, h = cv.boundingRect(coords) # Find minimum spanning bounding box
    rect = rotated[y:y+h, x:x+w] 
    rect = cv.cvtColor(rect, cv.COLOR_BGR2RGB)
    
    return rect

def torch_downsample(img, kernel_size, interpolation = InterpolationMode.BILINEAR):
    size = img.shape
    
    new_size = (size[0]//kernel_size, size[1]//kernel_size)
    
    tensor_img = np.moveaxis(img, -1, 0)
    tensor_img = torch.Tensor(tensor_img)
    
    resized_map = T.Resize(new_size, interpolation=interpolation)(tensor_img)
    
    resized_map = resized_map.numpy()
    resized_map = np.moveaxis(resized_map, 0, -1)
    
    return resized_map.astype(int)

def bilinear_interpolator(img, kernel_size):
    return torch_downsample(img, kernel_size, interpolation = InterpolationMode.BILINEAR)

def bilinear_interpolator_4x4(img):
    return bilinear_interpolator(img, 4)

def save_map_in_dir(map_dir):
    map_img = cv.imread(map_dir)
    
    logging.info("Rotating and cropping map...")
    fixed_map_img = rotate_crop_map(map_img)
    
    logging.info("Reducing file size...")
    fixed_map_img = bilinear_interpolator_4x4(fixed_map_img)
    
    os.remove(map_dir)
    Image.fromarray(np.uint8(fixed_map_img)).save(map_dir)
    

In [69]:
for i in range(10):
    print(f"Iteration: {i+1}")
    map_names = grouped_area_4.iloc[i]["IMAGE"]
    
    new_map_dir = os.path.join(MAP_DIRECTORY, str(i+1))
    os.makedirs(new_map_dir)
    
    for map_name in map_names:
        map_path = download_map_from_gdrive(str(map_name))
        save_map_in_dir(map_path)
        shutil.move(map_path, new_map_dir)

10-01-2023 14:08:52 INFO Fetching file 82878066.27.tif
10-01-2023 14:08:52 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782878066.27.tif%27&maxResults=1000&alt=json


Iteration: 1


10-01-2023 14:08:53 INFO Downloading file 82878066.27.tif
10-01-2023 14:08:53 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1rF6bS3v_RhA2ocFh7iorh7GXPV_0mSzI?alt=json
10-01-2023 14:10:35 INFO Done!
10-01-2023 14:10:41 INFO Rotating and cropping map...
10-01-2023 14:10:52 INFO Reducing file size...
10-01-2023 14:11:00 INFO Fetching file 82878069.27.tif
10-01-2023 14:11:00 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782878069.27.tif%27&maxResults=1000&alt=json
10-01-2023 14:11:00 INFO Downloading file 82878069.27.tif
10-01-2023 14:11:00 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1y9CFKVyd-M_SWSV1EdM-e9scYQjEpi5T?alt=json
10-01-2023 14:12:43 INFO Done!
10-01-2023 14:12:48 INFO Rotating and cropping map...
10-01-2023 14:13:01 INFO Reducing file size...
10-01-2023 14:13:11 INFO Fetching file 82878072.27.tif
10-01-2023 

Iteration: 2


10-01-2023 14:18:20 INFO Downloading file 82878024.27.tif
10-01-2023 14:18:20 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1r-OE99E0nG9QFo6ahjWx0uMnMDlTXFOv?alt=json
10-01-2023 14:20:08 INFO Done!
10-01-2023 14:20:13 INFO Rotating and cropping map...
10-01-2023 14:20:25 INFO Reducing file size...
10-01-2023 14:20:34 INFO Fetching file 82878027.27.tif
10-01-2023 14:20:34 INFO access_token is expired. Now: 2023-01-10 13:20:34.879330, token_expiry: 2023-01-10 13:19:17.713113
10-01-2023 14:20:34 INFO access_token is expired. Now: 2023-01-10 13:20:34.928506, token_expiry: 2023-01-10 13:19:17.713113
10-01-2023 14:20:34 INFO Refreshing access_token
10-01-2023 14:20:35 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782878027.27.tif%27&maxResults=1000&alt=json
10-01-2023 14:20:36 INFO Downloading file 82878027.27.tif
10-01-2023 14:20:36 DEBUG URL being r

...

10-01-2023 14:56:14 INFO Downloading file 82877610.27.tif
10-01-2023 14:56:14 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1MJfH1FgvKwJXt3gOvHGlzqSC1geT2HuL?alt=json
10-01-2023 14:57:21 INFO Done!
10-01-2023 14:57:24 INFO Rotating and cropping map...
10-01-2023 14:57:34 INFO Reducing file size...
10-01-2023 14:57:39 INFO Fetching file 82877613.27.tif
10-01-2023 14:57:39 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782877613.27.tif%27&maxResults=1000&alt=json
10-01-2023 14:57:40 INFO Downloading file 82877613.27.tif
10-01-2023 14:57:40 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1MnrnyZv95w_irLqC1KJ5XKp5GdGBctaZ?alt=json
10-01-2023 14:58:49 INFO Done!
10-01-2023 14:58:52 INFO Rotating and cropping map...
10-01-2023 14:59:02 INFO Reducing file size...
10-01-2023 14:59:07 INFO Fetching file 82877616.27.tif
10-01-2023 

Iteration: 9


10-01-2023 15:02:07 INFO Downloading file 82877571.27.tif
10-01-2023 15:02:07 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/19KiafGBbxTMvuuQ5LOu0yM5hVUWS2V3d?alt=json
10-01-2023 15:03:19 INFO Done!
10-01-2023 15:03:24 INFO Rotating and cropping map...
10-01-2023 15:03:32 INFO Reducing file size...
10-01-2023 15:03:40 INFO Fetching file 82877574.27.tif
10-01-2023 15:03:40 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782877574.27.tif%27&maxResults=1000&alt=json
10-01-2023 15:03:42 INFO Downloading file 82877574.27.tif
10-01-2023 15:03:42 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1mo21HOrjKEOsu1PUnXhf8R6W8gNthmwW?alt=json
10-01-2023 15:04:48 INFO Done!
10-01-2023 15:04:52 INFO Rotating and cropping map...
10-01-2023 15:05:06 INFO Reducing file size...
10-01-2023 15:05:13 INFO Fetching file 82877577.27.tif
10-01-2023 

Iteration: 10


10-01-2023 15:08:24 INFO Downloading file 82877529.27.tif
10-01-2023 15:08:24 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1GgLHYElY5RYOP8hUItjRG9yl9_09nCcE?alt=json
10-01-2023 15:09:31 INFO Done!
10-01-2023 15:09:33 INFO Rotating and cropping map...
10-01-2023 15:09:50 INFO Reducing file size...
10-01-2023 15:10:01 INFO Fetching file 82877532.27.tif
10-01-2023 15:10:01 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782877532.27.tif%27&maxResults=1000&alt=json
10-01-2023 15:10:02 INFO Downloading file 82877532.27.tif
10-01-2023 15:10:02 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1iGQ9W8LIUUzF_gTdBxRkW8Lrsy7f8m2t?alt=json
10-01-2023 15:11:10 INFO Done!
10-01-2023 15:11:13 INFO Rotating and cropping map...
10-01-2023 15:11:27 INFO Reducing file size...
10-01-2023 15:11:36 INFO Fetching file 82877535.27.tif
10-01-2023 

In [70]:
for i in range(10,20):
    print(f"Iteration: {i+1}")
    map_names = grouped_area_4.iloc[i]["IMAGE"]
    
    new_map_dir = os.path.join(MAP_DIRECTORY, str(i+1))
    os.makedirs(new_map_dir)
    
    for map_name in map_names:
        map_path = download_map_from_gdrive(str(map_name))
        save_map_in_dir(map_path)
        shutil.move(map_path, new_map_dir)

10-01-2023 15:16:39 INFO Fetching file 82877520.27.tif
10-01-2023 15:16:39 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782877520.27.tif%27&maxResults=1000&alt=json


Iteration: 11


10-01-2023 15:16:40 INFO Downloading file 82877520.27.tif
10-01-2023 15:16:40 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1aPUY5X8cGNEIPxDcfBimX38UcXVfuYmt?alt=json
10-01-2023 15:17:46 INFO Done!
10-01-2023 15:17:49 INFO Rotating and cropping map...
10-01-2023 15:18:03 INFO Reducing file size...
10-01-2023 15:18:16 INFO Fetching file 82877523.27.tif
10-01-2023 15:18:16 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782877523.27.tif%27&maxResults=1000&alt=json
10-01-2023 15:18:16 INFO Downloading file 82877523.27.tif
10-01-2023 15:18:16 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1Grzi86b4lO1IEsFJTIazjXylXIBB96QZ?alt=json
10-01-2023 15:19:32 INFO Done!
10-01-2023 15:19:34 INFO Rotating and cropping map...
10-01-2023 15:19:44 INFO Reducing file size...
10-01-2023 15:19:52 INFO Fetching file 87213030.27.tif
10-01-2023 

Iteration: 12


10-01-2023 15:24:00 INFO Downloading file 82877184.27.tif
10-01-2023 15:24:00 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/12bs6akbGCjtqmnzia1H7aTQ47MSyHOnw?alt=json
10-01-2023 15:25:08 INFO Done!
10-01-2023 15:25:10 INFO Rotating and cropping map...
10-01-2023 15:25:19 INFO Reducing file size...
10-01-2023 15:25:26 INFO Fetching file 82877187.27.tif
10-01-2023 15:25:26 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782877187.27.tif%27&maxResults=1000&alt=json
10-01-2023 15:25:26 INFO Downloading file 82877187.27.tif
10-01-2023 15:25:26 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1toSF8gj_gsjUED6ZTc_YM67c830eprY3?alt=json
10-01-2023 15:26:53 INFO Done!
10-01-2023 15:26:56 INFO Rotating and cropping map...
10-01-2023 15:27:08 INFO Reducing file size...
10-01-2023 15:27:15 INFO Fetching file 82877190.27.tif
10-01-2023 

...

10-01-2023 16:04:42 INFO Downloading file 82877361.27.tif
10-01-2023 16:04:42 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1RmieYuLh5akfMcLGr9V0xmOyIyDTSEWr?alt=json
10-01-2023 16:05:54 INFO Done!
10-01-2023 16:05:56 INFO Rotating and cropping map...
10-01-2023 16:06:09 INFO Reducing file size...
10-01-2023 16:06:19 INFO Fetching file 82877364.27.tif
10-01-2023 16:06:19 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782877364.27.tif%27&maxResults=1000&alt=json
10-01-2023 16:06:20 INFO Downloading file 82877364.27.tif
10-01-2023 16:06:20 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1oNqkVYgk4-R_ru6iWv7zuyehxaSoVXni?alt=json
10-01-2023 16:07:34 INFO Done!
10-01-2023 16:07:36 INFO Rotating and cropping map...
10-01-2023 16:07:49 INFO Reducing file size...
10-01-2023 16:07:54 INFO Fetching file 82877367.27.tif
10-01-2023 

Iteration: 19


10-01-2023 16:11:38 INFO Downloading file 82877205.27.tif
10-01-2023 16:11:38 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1upSeNZ4ErucNDElxi6P2XoEGFPuq6a7g?alt=json
10-01-2023 16:13:02 INFO Done!
10-01-2023 16:13:09 INFO Rotating and cropping map...
10-01-2023 16:13:25 INFO Reducing file size...
10-01-2023 16:13:35 INFO Fetching file 82877208.27.tif
10-01-2023 16:13:35 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782877208.27.tif%27&maxResults=1000&alt=json
10-01-2023 16:13:35 INFO Downloading file 82877208.27.tif
10-01-2023 16:13:35 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1-j7sJ1UmzkCIMWRrPZEv2FoNZbzlQJ94?alt=json
10-01-2023 16:14:58 INFO Done!
10-01-2023 16:15:00 INFO Rotating and cropping map...
10-01-2023 16:15:16 INFO Reducing file size...
10-01-2023 16:15:24 INFO Fetching file 82877211.27.tif
10-01-2023 

Iteration: 20


10-01-2023 16:19:18 INFO Downloading file 82877166.27.tif
10-01-2023 16:19:18 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1uJfjO_Tqe3RmuMJ06iw_ASJZb3sxgPg9?alt=json
10-01-2023 16:20:42 INFO Done!
10-01-2023 16:20:44 INFO Rotating and cropping map...
10-01-2023 16:21:30 INFO Reducing file size...
10-01-2023 16:21:34 INFO Fetching file 82877169.27.tif
10-01-2023 16:21:34 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782877169.27.tif%27&maxResults=1000&alt=json
10-01-2023 16:21:34 INFO Downloading file 82877169.27.tif
10-01-2023 16:21:34 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1lzGdZAByVrHcRpyDVf8UlnVLqa8SU-Y1?alt=json
10-01-2023 16:23:07 INFO Done!
10-01-2023 16:23:09 INFO Rotating and cropping map...
10-01-2023 16:23:56 INFO Reducing file size...
10-01-2023 16:23:59 INFO Fetching file 82877172.27.tif
10-01-2023 

In [ ]:
for i in range(20,30):
    print(f"Iteration: {i+1}")
    map_names = grouped_area_4.iloc[i]["IMAGE"]
    
    new_map_dir = os.path.join(MAP_DIRECTORY, str(i+1))
    os.makedirs(new_map_dir)
    
    for map_name in map_names:
        map_path = download_map_from_gdrive(str(map_name))
        save_map_in_dir(map_path)
        shutil.move(map_path, new_map_dir)

10-01-2023 16:29:48 INFO Fetching file 82877916.27.tif
10-01-2023 16:29:48 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782877916.27.tif%27&maxResults=1000&alt=json


Iteration: 21


10-01-2023 16:29:49 INFO Downloading file 82877916.27.tif
10-01-2023 16:29:49 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1eVVDfzVWP_NmaHmljb4EAO-C88tePwBY?alt=json
10-01-2023 16:31:19 INFO Done!
10-01-2023 16:31:25 INFO Rotating and cropping map...
10-01-2023 16:31:36 INFO Reducing file size...
10-01-2023 16:31:46 INFO Fetching file 82877919.27.tif
10-01-2023 16:31:46 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782877919.27.tif%27&maxResults=1000&alt=json
10-01-2023 16:31:47 INFO Downloading file 82877919.27.tif
10-01-2023 16:31:47 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1WtlgkyJkRXF7eH5DHsz_leAXbZd39J-D?alt=json
10-01-2023 16:33:50 INFO Done!
10-01-2023 16:33:52 INFO Rotating and cropping map...
10-01-2023 16:34:04 INFO Reducing file size...
10-01-2023 16:34:13 INFO Fetching file 82877922.27.tif
10-01-2023 

Iteration: 22


10-01-2023 16:38:55 INFO Downloading file 82877868.27.tif
10-01-2023 16:38:55 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1FC9hQ_Cl19ok7kIwCTZOHsTaizp152Bu?alt=json
10-01-2023 16:40:20 INFO Done!
10-01-2023 16:40:23 INFO Rotating and cropping map...
10-01-2023 16:40:34 INFO Reducing file size...
10-01-2023 16:40:49 INFO Fetching file 82877871.27.tif
10-01-2023 16:40:49 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%2713favbDFNNPK78LNSIiWRCzNyhFtYe1Ix%27+in+parents+and+trashed%3Dfalse+and+title+%3D+%2782877871.27.tif%27&maxResults=1000&alt=json
10-01-2023 16:40:49 INFO Downloading file 82877871.27.tif
10-01-2023 16:40:49 DEBUG URL being requested: GET https://www.googleapis.com/drive/v2/files/1LWrponXe7VjA1pnwL_6xsSpjL_em-Mh7?alt=json
10-01-2023 16:42:22 INFO Done!
10-01-2023 16:42:24 INFO Rotating and cropping map...
10-01-2023 16:42:34 INFO Reducing file size...
10-01-2023 16:42:46 INFO Fetching file 82877874.27.tif
10-01-2023 